<a href="https://colab.research.google.com/github/danny1461/CSCI-191T-Machine-Learning/blob/main/Feed_Forward_%26_Back_Propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feed Forward & Back Propagation
By Daniel Flynn

In [107]:
import random
import math

def linearSum(weights, inputs):
  return sum([w*i for w, i in zip(weights, inputs)])

def logisticalRegression(weights, inputs):
  return 1.0 / (1.0 + math.e ** (-linearSum(weights, inputs)))

class Node():
  def __init__(self, inputCount):
    self.alg = logisticalRegression
    self.weights = [random.uniform(-0.1, 0.1) for i in range(inputCount + 1)]

  def getOutput(self, inputs):
    self.output = self.alg(self.weights, inputs)
    return self.output

class Graph:
  def __init__(self, layers):
    self.layers = []
    self.deltas = []
    for i in range(len(layers) - 1):
      self.layers.append([ Node(layers[i]) for j in range(layers[i + 1]) ])
      self.deltas.append([0] * layers[i + 1])
    self.outputNdx = len(self.layers) - 1

  def getOutput(self, inputs):
    for layer in self.layers:
      inputs = [ node.getOutput([1] + inputs) for node in layer ]

    return inputs

  def calculateDeltas(self, expected):
    for i in reversed(range(len(self.layers))):
      layer = self.layers[i]

      if i == self.outputNdx:
        errors = [node.output - r for node, r in zip(layer, expected)]
      else:
        errors = []
        for j in range(len(layer)):
          error = 0.0
          for k in range(len(self.layers[i + 1])):
            error += self.layers[i + 1][k].weights[j + 1] * self.deltas[i + 1][k]
          errors.append(error)

      for j in range(len(layer)):
        node = layer[j]
        self.deltas[i][j] = errors[j] * node.output * (1.0 - node.output)

  def updateWeights(self, inputs, learningRate):
    for i in range(len(self.layers)):
      if i > 0:
        inputs = [node.output for node in self.layers[i - 1]]
        
      inputs = [1] + inputs
      for j in range(len(self.layers[i])):
        for k in range(len(inputs)):
          self.layers[i][j].weights[k] -= learningRate * self.deltas[i][j] * inputs[k]

  def train(self, trainingSet, learningRate = 0.5, learningThreshhold = 0.000001, displayRegularity = 1000, displayFn = None):
    last_error = 0
    error = float('inf')
    iterations = 0
    while last_error - error > learningThreshhold or math.isinf(error):
      last_error = error;

      error = 0
      for inputs, expected in trainingSet:
        outputs = self.getOutput(inputs)

        error += sum([(expected[i] - outputs[i])**2 for i in range(len(expected))])
        self.calculateDeltas(expected)
        self.updateWeights(inputs, learningRate)
      
      iterations += 1
      if iterations % displayRegularity == 0:
        print('Iteration {}: Error = {}'.format(iterations, error))
        if displayFn != None:
          displayFn()

    if iterations % displayRegularity != 0:
      print('Iteration {}: Error = {}'.format(iterations, error))
      if displayFn != None:
        displayFn()

def evaluateGraph(graph, trainingSet):
    stats = {}
    error = 0
    for inputs, expected in trainingSet:
      outputs = g.getOutput(inputs)
      error += sum([(expected[i] - outputs[i])**2 for i in range(len(expected))])
      key = (expected[0], 1 if outputs[0] >= 0.5 else 0)
      stats[key] = stats.get(key, 0) + 1
    
    print('Recal:', stats.get((1, 1), 0) / (stats.get((1, 1), 0) + stats.get((1, 0), 0)))
    print('Precision:', stats.get((1, 1), 0) / (stats.get((1, 1), 0) + stats.get((0, 1), 0)))
    print('')

def printGraph(graph, key = 'weights'):
  label = key.capitalize()
  for i in range(len(graph.layers[0])):
    print('w{} {}: '.format(i + 1, label), getattr(graph.layers[0][i], key))
  for i in range(len(graph.layers[1])):
    print('y{} {}: '.format(i + 1, label), getattr(graph.layers[1][i], key))

In [108]:
random.seed(2021)

# 2 Input Nodes
# 2 Hidden Nodes
# 1 Output Node
g = Graph([2, 2, 1])

print('Starting State...')
printGraph(g)
print('')

trainingData = [
  ( [0, 0], [0] ),
  ( [0, 1], [1] ),
  ( [1, 0], [1] ),
  ( [1, 1], [0] ),
]

g.train(trainingData, displayFn = lambda: evaluateGraph(g, trainingData))
print('Final State...')
printGraph(g)

Starting State...
w1 Weights:  [0.06726750046641483, 0.07166048084163751, 0.008822066004144327]
w2 Weights:  [-0.05053353327699653, 0.027004118315371473, 0.09472637093585565]
y1 Weights:  [-0.0052418342615191404, -0.08725176406969695, -0.04601109132754114]

Iteration 1000: Error = 1.0315529244237684
Recal: 0.5
Precision: 0.5

Iteration 2000: Error = 0.7041606836014094
Recal: 1.0
Precision: 0.6666666666666666

Iteration 3000: Error = 0.02666396980633983
Recal: 1.0
Precision: 1.0

Iteration 4000: Error = 0.008288459484369187
Recal: 1.0
Precision: 1.0

Iteration 5000: Error = 0.004761404737942236
Recal: 1.0
Precision: 1.0

Iteration 6000: Error = 0.00330913665554861
Recal: 1.0
Precision: 1.0

Iteration 6034: Error = 0.0032748063687582443
Recal: 1.0
Precision: 1.0

Final State...
w1 Weights:  [2.575335903705313, -6.477063954334737, -6.500597376693722]
w2 Weights:  [6.48388163444038, -4.378804740383171, -4.38084896165839]
y1 Weights:  [-4.166028593693109, -9.012724897902848, 8.8868511388452

In this case, we're able to get convergence. But I have seen some random starting weights that seem to never converge. I'm not sure if that's because my learning rate is too high or if there is a local minimum there that the code can't get out of.

So as it happens, my starting seed of the current year (2021) succeeds.

### Graph state for each input

In [111]:
log = True
for inputs, expected in trainingData:
  print('Inputs: ', inputs)
  g.getOutput(inputs)
  print('Graph State...')
  printGraph(g, key = 'output')
  print('Expected Output: ', expected)
  print('')

Inputs:  [0, 0]
Graph State...
w1 Output:  0.9292572730308457
w2 Output:  0.9984744622939888
y1 Output:  0.02489508095471664
Expected Output:  [0]

Inputs:  [0, 1]
Graph State...
w1 Output:  0.019354967001063178
w2 Output:  0.8911975892963746
y1 Output:  0.9728663147768487
Expected Output:  [1]

Inputs:  [1, 0]
Graph State...
w1 Output:  0.01980672877570542
w2 Output:  0.8913956476249763
y1 Output:  0.9728052306072238
Expected Output:  [1]

Inputs:  [1, 1]
Graph State...
w1 Output:  3.0360874889983035e-05
w2 Output:  0.09314948399027904
y1 Output:  0.034273883186918354
Expected Output:  [0]

